# This is a sandbox for setting up a run of the simulation on `PRIVATE_AUTO` only. 

* It can be run on a small subsample pandas DataFrame
* The dask implementation might work with `map_partition()` but also might not. 


In [7]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import os
import joblib

%reload_ext autoreload
%autoreload 2



pd.set_option('display.max_columns', None)

datadir = '../../data'
mode = 'PRIVATE_AUTO'
simulation_id = 'baseline'
test = True

#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 
if test:
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = df.loc[df['mode'] == mode]
    df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
    simulation_id = 'test'

else: 


    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]



In [8]:
df

,index,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
0,4199,7952205927434267893,10000366922410938523,PRIVATE_AUTO,HOME,EAT,0 days 12:41:15,0 days 12:43:31,0.602554,nan,530330068002,47.636260,-122.358964,530330067004,47.635745,-122.347078,commercial,retail,commercial,retail,residential,multi_family,residential,multi_family,47.638249,-122.356634,47.636388,-122.346274,thursday,1732374551127832797,530330067004,nan,nan,53033006700,nan,nan,35_64,62.0,M,white,not_hispanic_or_latino,20000_40000,22176.0,notinlf,some_college,not_attending_school,naics72,head_of_household,0,nan,not_working,renter,same_house,5_person,40000_75000,59308.0,nonfamily_single,1,multiple_units,core,english,47.636388,-122.346274,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME
1,3575,506829784905170377,10001830118790204821,PRIVATE_AUTO,HOME,EAT,0 days 19:43:00,0 days 19:56:05,1.011157,nan,530610516012,47.844719,-122.299453,530610516021,47.834790,-122.308797,commercial,retail,commercial,retail,mixed_use,single_family,residential,single_family,47.840838,-122.300091,47.834503,-122.313617,thursday,3635700334446387909,530610516021,530350805002,530610516014,53061051602,53035080500,53061051601,18_24,18.0,M,white,not_hispanic_or_latino,0_20000,216.0,employed,high_school,school,naics722513,child,0,nan,carpool,renter,same_house,3_person,40000_75000,41273.0,family_single,2,multiple_units,core,english,47.834503,-122.313617,NaN,NaN,47.845652,-122.315406,worked_in_person,HOME
2,4816,13722390883687412311,10002290586277908152,PRIVATE_AUTO,HOME,EAT,0 days 18:40:12,0 days 18:46:32,2.150550,nan,530330323092,47.675673,-122.126007,530330228023,47.663252,-122.138520,mixed_use,office,commercial,retail,residential,single_family,residential,single_family,47.670172,-122.111026,47.664865,-122.135813,thursday,4129290245843427363,530330228023,nan,nan,53033022802,nan,nan,25_34,34.0,F,asian,not_hispanic_or_latino,lte_0,0.0,notinlf,advanced_degree,not_attending_school,not_working,spouse,0,nan,not_working,owner,same_house,5_person,125000_plus,319710.0,married_couple,3_plus,single_family,core,asian_pacific,47.664865,-122.135813,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME
3,247,11325025596515485190,10002759413662457525,PRIVATE_AUTO,WORK,HOME,0 days 07:24:41,0 days 07:37:33,6.867275,nan,530670109103,47.016462,-122.909304,530670112003,47.047701,-122.808723,residential,single_family,residential,single_family,civic_institutional,education,civic_institutional,education,47.016530,-122.908200,47.040000,-122.800000,thursday,3706854826325843051,530670109103,530670112003,nan,53067010910,53067011200,nan,25_34,33.0,F,white,not_hispanic_or_latino,40000_80000,51862.0,employed,high_school,not_attending_school,naics92,spouse,0,nan,driving,owner,same_house,4_person,75000_125000,111288.0,married_couple,3_plus,single_family,core,other,47.016530,-122.908200,47.040000,-122.80000,NaN,NaN,worked_in_person,WORK
4,4243,2725037000145676387,10002759413662457525,PRIVATE_AUTO,SHOP,WORK,0 days 09:52:13,0 days 09:58:59,1.866797,nan,530670112003,47.047701,-122.808723,530670112003,47.047701,-122.808723,civic_institutional,education,civic_institutional,edu

run the simulation on small dataset to test. 

In [ ]:
# this takes ~1min for 10,000 trips
# out = sim.simulate_person_load(df, existing_load, 'base', managed=False)

embarrassingly parallel

In [13]:
number_of_chunks = 10
df_list = np.array_split(df, number_of_chunks)

In [24]:
# run the simulation in parallel
# df must be a pandas dataframe
charge_sims = joblib.Parallel(verbose=10, n_jobs=-1)(
    joblib.delayed(sim.simulate_person_load)(
    trips_df=df,
    existing_load=existing_load,
    simulation_id=simulation_id,
    managed=False
) for df in df_list)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    5.2s remaining:   12.1s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    5.3s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    5.3s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.5s finished


### Restack the DataFrames

In [18]:
# restack the dataframes
charges_list = [x['charges'] for x in charge_sims]
loads_list = [x['loads'] for x in charge_sims]

In [21]:
charges_list

AttributeError: 'list' object has no attribute 'compute'

Pandas significantly faster since we can fit these in memory. 

In [16]:
# restack the dataframes
charges_df = pd.concat(charges_list)
loads_df = pd.concat(loads_list) # huge ~200 million rows

TypeError: cannot concatenate object of type '<class 'dask.delayed.Delayed'>'; only Series and DataFrame objs are valid

In [ ]:
loads_df

,person_id,load_segment_id,charge_id,window_start_time,window_end_time,load_kW
0,1000016319449981116,7318818662359676746_base_0,7318818662359676746_base,0 days 00:00:00,0 days 01:00:00,7.200000
1,1000016319449981116,7318818662359676746_base_1,7318818662359676746_base,0 days 01:00:00,0 days 02:00:00,7.200000
2,1000016319449981116,7318818662359676746_base_2,7318818662359676746_base,0 days 02:00:00,0 days 03:00:00,1.647283
3,1000016319449981116,7318818662359676746_base_3,7318818662359676746_base,0 days 03:00:00,0 days 04:00:00,0.000000
4,1000016319449981116,7318818662359676746_base_4,7318818662359676746_base,0 days 04:00:00,0 days 05:00:00,0.000000
...,...,...,...,...,...,...
19,9999319521751741792,1701538494003822315_base_19,1701538494003822315_base,0 days 19:00:00,0 days 20:00:00,0.000000
20,9999319521751741792,1701538494003822315_base_20,1701538494003822315_base,0 days 20:00:00,0 days 21:00:00,0.000000
21,9999319521751741792,1701538494003822315_base_21,1701538494003822315_base,0 days 21:00:00,0 days 22:00:00,0.000000
22,9999319521751741792,1701538494003822315_base_22,1701538494003822315_base,0 days 22:00:00,0 days 23:00:00,0.000000


In [ ]:
charges_df.to_parquet(os.path.join(datadir, f'charges_{mode}_{simulation_id}.parquet'), overwrite=True)
loads_df.to_parquet(os.path.join(datadir, f'loads_{mode}_{simulation_id}.parquet'), overwrite=True)

NameError: name 'charges_df' is not defined